# Applicative Functors?

Applicative functors are, let me say, a desin pattern, popular in typed functional languages. Mostly Haskell, because of their roots in cateory-theory and that sounds like a thing people writing Haskell like. Based on conference talks Scala people are trying to use this pattern as well. So my question would be, is this still usefull even if I don't have a static type system? 

# Why would anybody deal with Category Theory in programming?

Well, to understand that, you need to imagine yourself in the shoes of a beginner haskell programmer. You have your awesome functional language, and really like the predictability.

* you can't change values
* no null/nil/None/undefined or any such nonsense
* this means there is no global state, awesome
* all functions are pure, stateless, defined only by relation of inputs to outputs, no I/O
 * basically the most testable thing ever

And all of this is awesome, but after a while you realize that not having I/O kind-of sucks, and thar what you have is basically a really tricked-out command-line calculator. (At least that was my experience learning Haskell at my uni)

The question then became, how do we keep the awesome things about Haskell, like no Null-pointer exceptions, testable stateless functions, but add new things on top of it. It often makes sense to have global variable, I/O, or return null. If want to build i.e. a database, all of these look useful. 

What then happened was that Philip Wadler read a paper about using monads from category theory as interface to chain I/O operations. And he implemented this as the base for I/O in haskell. And 25 years later we see monadic interfaces in C# Linq and Tasks, Java Streams and almost in Javascript Promise.

I would like to show few more things out of this algebraic cookbook :) 

# Who should read this?

Anybody who is interested with using algebraic design patterns in Python.
I have three kinds of people in mind as my target audience.

* Me :-) Lets be clear, similiar to many monad tutorials, this is mostly a record for my future self, to remind me that I once understood something about this. 
* People familliar with these concepts in haskell, wanting to see them in dynamically typed language like Python
* People interested in these algebraic patterns that know Pthon and don't have resources to learn i.e. haskell.

If you are somebody interested, but wants to see these in haskell rather than python, go read [Bartosz Milewski's Programming Cafe](https://bartoszmilewski.com/), or [haskellforall by Gabriel Gonzalez](http://www.haskellforall.com/).

As with the previous article, some of my examples will be attempts to translate some articles about haskell to python :)

# Introducing Functor

Category theory is usually concerned with relation between diferent categories, but for programming, we are usually concerned only with category of types and the unary functions that convert between them.

When programmer talks about relarion between two categories,
usually one will basically be his favourite language and second one
some fancy extension to that language.

Like said original Haskell and Haskell with I/O.

So what in practice is a fuctor?

In general, anything where you can define a sensible `map` operation.
For example:

* list
* stream
* generator
* tree

Sensible for list means, that it shouldn't matter whether you
strs.map(trim).map(uppercase)

or

strs.map(lambda x: uppercase(trim(x)))

# Do you even lift?

There is a recuring concept, where we use some bit of information about
our datatype to convert functions that know nothig about it, to work with it.

We call this *lifting*.

For example, because I know that lists have map function, I could lift i.e. `str` to work on lists of things.


In [ ]:
def lift(fn):
    def liftedfn(x):
      return list(map(fn,x))
    return liftedfn 

In [ ]:
lifted_str = lift(str)
lifted_str([1,2,3])

# Category theory!

This idea of translating objects from one domain to another is at the core of the mathematical subject of category theory. 

Category in mathemathics is defined by three things:
* collection of objects
* collection of possible transformations between these objects, called morphisms
* if we can transform object X to Y, and we can transform object Y to Z, there needs to be transformation from A to C in our transformation collection

You could draw a category using upper-case letters for objects, and arrows to signify possible transformations.

![Category](https://upload.wikimedia.org/wikipedia/commons/e/ef/Commutative_diagram_for_morphism.svg)

Functor is then mapping between categories. On the image you can see a category of red objects, with morphisms $f$, $g$ and three identity morphisms (i.e. arrows that transform and object to itself) and a functor $F$ that *lifts* the transformations from red category, to green, where there are only two objects.

![Functor](https://upload.wikimedia.org/wikibooks/en/3/36/Functor.png) 

# What if I have multiple parameters?

Mapping only single param functions can be limiting. It would be cool if `lift` would could work on function with arbitrary many parameters.

Could *currying* rescue us?

Currying is a concept, where instead of passing in all the params into the function at once, you pass in the first param and grt a new function as a result. This function accepts the second param and returns a function that accepts the third, and by now you should be getting a picture.

Or maybe I should show an example.Lets invent a nontrivial 3 arg function, we will call it erm, `window` (because slice was taken).

In [3]:
from functools import partial

def window(begin,end,array):
    return array[begin:end]

window(2,4,[1,2,3,4,5])

[3, 4]

If this function were curried, it would look like this:

In [ ]:
def curried_window(begin):
    return lambda end: (lambda array: array[begin:end])
curried_window(2)(4)([1,2,3,4,5])

Some functional languages have this as a default. The nice thing about that is, that you can quickly create more specific fuctions, without the need to wrap them in new function definitions.

And this seems to solve out problem with multi param function, because sudenly, everything is a single param function, if you squint hard enough :)

In [ ]:
list(map(curried_window,[1,2,3,4]))

Ok, as you can see, currying won't save us just yet, because even though we can use map with them,
result is kind-of useless list of functions. We would need to somehow be able to *apply* this list of functions on a list of values. 

In [ ]:
def apply(lfn,lx):
    return [fn(x) for fn in lfn for x in lx]


a0 = map(curried_window,[1,2])
a1 = apply(a0,[5,6])
a2 = apply(a1,[
    [1,2,3,4,5,6],
    [1,2,3,4,5,6]
              ])

a2

Yay, this seems to have worked :)

Now we can try to combine currying with *apply* and create a propper lift function :)


In [5]:
def curry(n, fn):
    if n == 1:
        return fn
    if n == 2:
        return lambda x:partial(fn,x)
    else:
        return lambda x:curry(n-1,partial(fn,x))
    
def lift(fn):
    def lifted(arg0, *args):
        result = map(curry(len(args)+1, fn),arg0)
        for a in args:
            result = apply(result, a)
        return result
    return lifted

There is one more assumption we made, when creating this lift function.
On the line where we state:
```
result = map(curry(len(args)+1, fn),arg0)
```
we assume that you can use `map` function on the data-type we are lifting.

In [ ]:
liftedw = lift(window)
liftedw([1,2], [5,6], [
    [1,2,3,4,5,6],
    [1,2,3,4,5,6],
])

So, why to through all of this trouble to get the equivalent of i.e:

In [ ]:
[window(x,y,z) 
 for x in [1,2] 
 for y in [5,6] 
 for z in [[1,2,3,4,5,6],[1,2,3,4,5,6]]]

I have two reasons:
* We could use this pattern for things that are not iterable :)
* we can analyze the pattern in isolation and figure out interesting relations

# Lifting values

So far, we were concerned about lifting functions, but we would probably like to be able to lift arbitrary values as well. As you can see in our `lifted` definition, we have so far assumed, that we are lifting at least an unary function. Because we can't derive the wrapper for values from map and apply, we need to define it for the specific datatype. We uaually call this function `pure`

In [ ]:
def pure(value):
    return [value]

Interesting thing with `pure` is, that with it defined, we should be able to replace uses of `map(fn,o)` with `apply(pure(fn),o)`.

# Algebra behind the applicative

So far we have not been explicit about our formal requirements on `map`, `applicative` and `pure`. I have mentioned that `map` should behave *sensible* and that there is some relationship between functors, lifting and category theory.

In my mind, the most sensible thing, is to think about functor as a lawful interface. Lawful interface would mean, that it conforms to more requirements, than just the ability to i.e. perform a method call.

A type T describes a functor if:
* we can consider T a wrapper over arbitrary type A
* there is a function `map`, that
  * accepts function `f`, that accepts `a` and returns `b`
  * accepts `T` wrapping `a`
  * returns `T` wrapping `b`
  
In haskell we would say that map has type (a -> b) -> T a -> T b
With [Python's typing library](https://docs.python.org/3/library/typing.html) we could say that the type of `map` is
```
Callable[[Callable[[A],B], T[A]],T[B]]
```

The map function then needs to conform to these laws
* for any t, `map(lambda x:x ,t) == t`
* for any t and any two composable functions f and g, `map(f,map(g,t)) == map(lambda x: f(g(x)),t)`

If we define `id` and `compose` functions:
```
def id(x):
  return x
  
def compose(f,g):
  def composed(x)
    return f(g(x))
  return composed
```

we can swap the lambda expressions in the laws:
* for any t, `map(id ,t) == t`
* for any t and any two composable functions f and g, `map(f,map(g,t)) == map(compose(f,g),t)`


For applicative, we need it to implement be functor as well as
* have function `pure` that accepts arbitrary `a` and returnt `T a`
* have function `apply`, that
  * for any function f, that accepts `a` and returns `b`
  * accepts `T` wrapping `f`
  * accepts `T` wrapping `a`
  * returns `T` wrapping `b`
  
In haskell, we would say, that `pure` has type a -> T a and apply T (a -> b) -> T a -> T b
With [Python's typing library](https://docs.python.org/3/library/typing.html) we could say that the type of `pure` is
```
Callable[[A], T[A]]
```
And type of `apply` is
```
Callable[[T[Callable[[A],B]], T[A]],T[B]]
```

The `apply` and `pure` functions need to conform to these laws:
* for any t, `apply(pure(id),t) == t`
* for any function f and type x, `apply(pure(f),pure(x)) == f(x)`
* for any t, x, `apply(t, pure(x)) == apply(pure(lambda g: g(x)),t)`
* for any u,v,w, `apply(apply(apply(pure(compose),u),v),w) == apply(u,apply(v,w))`
  * be aware that apply is not required to be associative, i.e. you can't state what ?? should be in `apply(apply(u,v),w) ?? apply(u,apply(v,w))`

# Why should we care about lawfulness?

If you look at the laws for both functor and applicative, they may seem arbitrary. Well, for functor, I have some intuition, because I had abstract algebra course at university and when I look at the laws my mins goes "This kida looks like definition of homomorphism, I remember those to be useful, I guess it makes sense then." I dont really remember the specifics, but because it is vaguely familliar, I am fine with it.

There is better reason to like lawful interfaces, though. Just by using the the laws of the interface, we can define new and usefull stuff.

Lets look at nesting of two different functors, where we would have T[U[A]]. It is easy to see, that for any a, T[U[a]] is a functor as well.

If map for T is tmap, and map for U is umap, we can define the tumap for the nesting of U in T like this:

```
def tumap(fn,tu):
  return tmap(lambda x: umap(fn,x), tu)
```

And because we know the functor laws we can try to prove that they hold for this implementation as well.

* for any `tu`, `tumap(id, tu) == tu`

$$ \begin{align*} \text{map}_{t_u}(id, t_u) &= {map}_{t} ( \lambda x: map_u(id,x), t_u) \\
&= {map}_{t} ( \lambda x: x, t_u) \\
&= {map}_{t} ( id, t_u) \\
&=  t_u \\
\end{align*} $$

* for any t and any two composable functions f and g, tumap(f,tumap(g,tu)) == map(compose(f,g),tu)
$$ \begin{align*} \text{map}_{t_u}(compose(f,g), t_u) &= {map}_{t} ( \lambda x: map_u(compose(f,g),x), t_u) \\
&= {map}_{t} ( \lambda x: map_u(compose(f,g),x), t_u) \\
&=  tu \\
\end{align*} $$


$$ \begin{align*} \text{map}_{t_u}(f, {map}_{tu}(g,tu)) &= {map}_{t} ( \lambda x: map_u(f,x), {map}_{t} ( \lambda x: map_u(g,x), tu)) \\
&= {map}_{t} ( \lambda x: x, tu) \\
&= {map}_{t} ( id, tu) \\
&=  tu \\
\end{align*} $$

# Parametrized lift

For some of the following examples, I need to be able to lift into different domains, so I am changing the lift function to be parametrized.

Because our these datatypes might not have `map` available, I am adding one more function to the mix, that is called `pure`. It is the constructor for the applicative datatype, that wraps a single value.

For list, pure function would be a singleton value:
```
def pureList(value):
  return [value]
```

The interesting thing is, that you can define `map` in terms of `apply` and `pure`:

```
def map(f, l):
  return apply(pure(f),l)
```


In [ ]:
def lift(pure, apply, fn):
    def lifted(arg0, *args):
        result = apply(pure(curry(len(args)+1, fn)),arg0)
        for a in args:
            result = apply(result, a)
        return result
    return lifted

# Solving the null-pointer exception?

One thing we could try, is to solve the NPE problem with this. If you think about it, when you have function, that doesn't expect `None`, but gets it, only reasonable thing we could do with this, is return `None`. 

We can define our wrapper function `pure` to just return the value back. We wouldn't be able to do this kind of cheating in haskell, where we would need to explicitly box the type. In Python this seems to be working.

We define our apply function to return `None` if any of the params are `None`.
We can try this on `int` function.

In [ ]:
def pureDefault(val):
    return val

def applyNone(lfn,lx):
    if lfn!=None and lx!=None:
        return lfn(lx)
    return None

nint = lift(pureDefault, applyNone, int)

Now we can compare:

In [ ]:
print(int("1"))
print(int(None))

In [ ]:
print(nint("1"))
print(nint(None))

To be honest, this is sort of a toy example. You probably don't want to just wrap your functions with this and instead of error at the place of failure just recieve the propagated nulls everywhere. But we can do better.  

# Applicative Validation

So what about some other uses? One I really like is using applicatives for validation.

First, let me harp on a thing I don't like about *exceptions*. Once you throw it,
you are aborting the execution. I especially dislike it, if somebody uses exceptions
to validate function params, i.e.:

In [ ]:
def validated_window(begin, end, arr):
    if begin < 0:
        raise Exception("Begin was negative")
    if end < 1:
        raise Exception("End was not positive")
    return window(begin,end,arr)

validated_window(-1,-1,[])

Exception only told me about the frst problem. But with applicative pattern, I can rewrite it in such way, that I would get all of them.

I will define two validating functions:

In [ ]:
def valid_begin(begin):
    return begin if begin >= 0 else Exception("Begin was negative")

def valid_end(end):
    return end if end > 0 else Exception("End was not positive")

Now, let me define how would I like to write the validated function.

In [ ]:
def validated_window(begin, end, arr):    
    return liftedw(valid_begin(begin),valid_end(end),arr)

Now the interesting part of the puzzle is to redefine applyy once again, from thing that processes values in arrays, to thing that can deal with values that might be useless Exceptions.

In [ ]:
def applyError(lfn,lx):
    if isinstance(lfn,Exception) and isinstance(lx,Exception):
        return Exception(*lfn.args,*lx.args)
    if isinstance(lfn,Exception):
        return lfn    
    if isinstance(lx,Exception):
        return lx
    return lfn(lx)
    

As you can see, there are 3 cases to consider:
* if both *fn* and *x* contain exceptions, I return agregate exception
* if *fn* or *x* contain exceptions, I return the exception
* if none of these happen, I can return *fn(x)*

In [ ]:
liftedw = lift(pureDefault, applyError, window);
validated_window(-1,-1,[])

This style of applicative validation is more or less taken from the [Purescript book](https://leanpub.com/purescript/read#leanpub-auto-applicative-validation). There is a difference, in the explicit boxing and unboxing I have mentioned earlier. If we explixitly boxed i.e. the `Exception` in an object of i.e. `Validated` class, we might be able to use some nice features of python, like sinlge dispatch. On the other hand, the lack of static type checking means, that the boxing wouldn't really help us.

# Composing applicatives

What we could do, on the other hand, is to create an class for defining applicatives:

In [2]:
class Applicative:
    def __init__(self, pure, apply):
        self.pure = pure
        self.apply = apply
        
    def lift(self, fn):
        def lifted(arg0, *args):
            result = self.apply(self.pure(curry(len(args)+1, fn)),arg0)
            for a in args:
                result = self.apply(result, a)
            return result
        return lifted

Now we can manipulate these. For example, I might want to be able to lift not only to domain of lists or domain of validated values, but some combination, like validated lists, or lists of validated values, or even validated lists or validated values.

In [ ]:
def containing(outer, inner):
    def pure(val):
        return outer.pure(inner.pure(val))
    def apply(lfn, lval):
        return outer.lift(inner.apply)(lfn,lval)
    return Applicative(pure,apply)

In [ ]:
ErrorAp = Applicative(pureDefault,applyError)

def pureArr(val):
    while True:
        yield val

def applyZip(lfn,lx):
    ar = zip(lfn, lx)
    return (fn(x) for fn, x in ar)

ListApp = Applicative(pureArr, applyZip)

ListOfValid = containing(ListApp, ErrorAp )

In [ ]:
list(ListOfValid.lift(window)([1,2], [5,6], [
    [1,2,3,4,5,6],
    [1,2,3,4,5,6],
]))

In [ ]:
list(ListOfValid.lift(window)([1,Exception("Begin was negative")], [5,Exception("End was not positive")], [
    [1,2,3,4,5,6],
    [1,2,3,4,5,6],
]))



In [ ]:
ListOfValid.lift(window)([1,Exception("Begin was negative")], [5, 6], [
    [1,2,3,4,5,6],
    [1,2,3,4,5,6],
])

# Reactive programming!

In [6]:
from ipywidgets import *
from IPython.display import display

a = IntSlider()
a.observe(lambda x: print(x['new']))

class Propagated:
    def __init__(self, last):
        self.last = last
        self.subscriptions = []
        
    def getValue(self):
        return self.last
    
    def subscribe(self, cb):
        self.subscriptions += [cb]
    
    def observe(self, prop):
        prop.subscribe(lambda x: self.set(x))
    
    def setValue(self, value):
        self.last = value
        for cb in self.subscriptions:
            cb(value)
    
def purePropagated(val):
    return Propagated(val)
    
def applyPropagated(fn,val):
    result = Propagated(fn.getValue()(val.getValue()))
    fn.subscribe(lambda f: result.setValue(f(val.getValue())))
    val.subscribe(lambda v: result.setValue(fn.getValue()(v)))
    return result

propApplicative = Applicative(purePropagated, applyPropagated)    
    
    
def pipeToPropagated(widget,propagated):
    def update(x):
        if x['name'] == 'value':
            propagated.setValue(x['new'])
    widget.observe(update)

def wrapWidgetInPropagated(widget):
    display(widget)
    p = Propagated(widget.value)
    pipeToPropagated(widget,p)
    return p;
    
v0 = wrapWidgetInPropagated(IntSlider())
v1 = wrapWidgetInPropagated(IntSlider())
s3 = IntSlider()

display(s3)

def add(a, b):
    return a + b

v3 = propApplicative.lift(add)(v0, v1)
def sub(x):
    s3.value = x
    
v3.subscribe(sub)